In [2]:
import os
import os.path as op
import glob
import pandas as pd

wd = "/mnt/scgc/simon/simonsproject/gss_vs/diamond/"

In [3]:
final_results = glob.glob(op.join(wd, '*_tbl.csv'))

In [4]:
len(final_results)

5015

In [5]:
intermediates = glob.glob(op.join(wd, "*LineP-all.daa"))

In [36]:
len(intermediates) - len(final_results)

56

In [6]:
isags = [op.basename(i).split("_")[0] for i in intermediates]
fsags = [op.basename(i).split("_")[0] for i in final_results]

In [7]:
incomplete = [i for i in isags if i not in fsags]

In [8]:
from viruscope.recruit import read_count, construct_recruit_tbl

In [9]:
vir_mg='/mnt/scgc_nfs/ref/viral_dbs/POV.fasta.gz'
bac_mg='/mnt/scgc_nfs/ref/viral_dbs/LineP-all.fasta.gz'

In [10]:
read_count_dict = {}
read_count_dict['vir_reads'] = read_count(vir_mg)
read_count_dict['bac_reads'] = read_count(bac_mg)

In [11]:
read_count_dict

{'bac_reads': '8279226', 'vir_reads': '5922080'}

In [12]:
protfile = lambda sag: op.join("/mnt/scgc/simon/simonsproject/gss_vs/prodigal/",'{sag}_proteins.fasta'.format(sag=sag))    
contigfile = lambda sag: op.join("/mnt/scgc/simon/simonsproject/gss_contigs/","contigs",'{sag}_contigs.fasta'.format(sag=sag))
rtsv = lambda sag, mg: op.join("/mnt/scgc/simon/simonsproject/gss_vs/diamond/{sag}_proteins_vs_{mg}.tsv.gz".format(sag=sag, mg=mg))
btsv = lambda sag: rtsv(sag, "LineP-all")
vtsv = lambda sag: rtsv(sag, "POV")

In [13]:
exsag = incomplete[0]

In [16]:
exdf = construct_recruit_tbl(vtsv(exsag), btsv(exsag), read_count_dict, contigfile(exsag))

doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.
looks like input config fiel is in fasta format.


KeyError: 'AG-414-G13_NODE_10_length_4263_cov_13.1659_ID_19'

In [20]:
from __future__ import print_function


import os
import os.path as op
import subprocess
import sys
import tempfile
import pandas as pd
from pysam import FastxFile
from viruscope.tools import file_transaction, file_exists

from viruscope.recruit import import_diamond_tsv, map_orfs_to_contigs, summarize_by_contig, contig_lengths, compute_fr

In [33]:
vir_tsv = vtsv(exsag)
bac_tsv = btsv(exsag)
contig_file = contigfile(exsag)
prot_file = protfile(exsag)

In [22]:
cnames = "qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore".split()
bac_df = import_diamond_tsv(bac_tsv)
vir_df = import_diamond_tsv(vir_tsv)

In [25]:
bac_sum = pd.Series(bac_df.groupby('sseqid')['qseqid'].count(), name='hit_mg-bac')
vir_sum = pd.Series(vir_df.groupby('sseqid')['qseqid'].count(), name='hit_mg-vir')

In [26]:
orfhits = pd.concat([bac_sum, vir_sum], axis=1).reset_index().rename(columns={'index':'orf'})
orfhits = map_orfs_to_contigs(orfhits, contig_file)

doesn't look like input contig file is in gff format.  Will assume that contig name is embedded in the ORF name.


In [28]:
chits = pd.concat([summarize_by_contig(orfhits, 'hit_mg-bac'), summarize_by_contig(orfhits, 'hit_mg-vir')], axis=1)
chits['reads_mg-vir'] = float(read_count_dict['vir_reads'])
chits['reads_mg-bac'] = float(read_count_dict['bac_reads'])

In [29]:
clens = contig_lengths(contig_file)

looks like input config fiel is in fasta format.


In [32]:
!head {contig_file}

>AG-414-G13_AG-414-G13_NODE_1
TGCAATCCCTATCCAAATAAATGTTTGTGATGTAGATATTGCAAAAGTATCAAATCCCTCATCACTCAATGCTTGAATCA
GACTCCATGCAAAAAAGATTCCTAAACCAGTTCCTAAAACGGCTCCAAATATTGAGATAATTGAAGATTCTATAAATATC
ATTCTCCTTATTTGTTTTCTATAGGTACCAATTGCTCTCATTAAACCTATTTCTCTTGTTCTTTCATAAACAGAAAGAGA
AAGTGTGTTTGTTATTCCAAATAATGCAACAAAAATTGATATTGATAAAAATCCGTAAATAACATTTAGAAGCTGTTGTA
TTTGAGTATTTGCTTCCTCAACTAAACCATCTTGATCTCTTAATTTAGCACCAGGATAGTCAGAAACAATTTGGTCTAAT
TTTGCTTGAGTTGATTCATTTTTTTCTTTTACATTAAAATAAATTTCTGTATCTAAAGATTCTTCTGAAAAGAATGAATA
ATTTTCTAATAGTAAAAAGAATTCAGCAGGAGGAGGGGTTGTCCAATCAAAAATATATTCAATAGTAAAGGTTCTTTTCC
CTTCTTCTGGGATTGTTAAGATTATCTCATCTCCAATACCAAGATTGTCTCTTTCTGCTTTCTGTTTAAGAACACCCATA
GCGTTTGGTTTCAAAAACTCTTCTTTTGAGCCAGCTATATTTTCTGTTTTTATTAAATCAAATATATCTTCATCAACCGC


In [34]:
!head {prot_file}

>AG-414-G13_NODE_1_length_335221_cov_44.5995_ID_1_1 # 1 # 2568 # -1 # ID=1_1;partial=10;start_type=ATG;rbs_motif=None;rbs_spacer=None;gc_cont=0.346
MLGFTKRPLFLIAWKNLKTYPVRILLTTSSIILGVSVIIASSIFSESNKSAFDNLFSGIY
EGIDLVVNPVQEDFAEGFGDSRGQGPLSFELKKIPDKRVQEVKDLPGVKDAWGEVFGFAQ
YIKIVRTECRSHSYERDCTETIFISNGFAPTFGGSWDTSPYAKQWELIDGKAPENNKEVV
MDRTTAENNNYSVGDRVTVLAGATPASFSIVGIADFANVGSPGGATFALFELKTAQTLLD
SRGQVDLINVVIEDNADIEAVRASIENIDKGTLDVINAQEAAAEQADGIKQGLNFFNTIL
NVFAGIALFVGAFIIQNTFRILLLQRTKELSLLRALGTSKNQIYRLVLSESIFMSLIGSG
LGIGLGIGLAVAVKEGLKYFEFGLPDGPLVLTTEAAISGAIVGITVTILSSLLPARKASQ
VSPMEAIRESTLTPKRKSLIKRLIFGTTVTTFGFAMLFGVLYDFLDLPTLSGLQQVGLGA
GVIFIGVSIITPSITKPFVFLFDKLYEIAFGILGKLSTENSKRTPRRTASTASALMIGLT


In [30]:
out_tbl = compute_fr(chits.reset_index(), clens, mult=1e6)

KeyError: 'AG-414-G13_NODE_10_length_4263_cov_13.1659_ID_19'

In [ ]:
def compute_fr(tbl, clens, mult=1e6):
    '''
    Args:
        tbl: output stats table with mg hit and read counts from diamond recruitment
        clens: dict of contig lengths
        mult: factor to multiply fraction by to make readiable

    Outputs:
        pandas DataFrame with mg_fr values calculated
    '''
    try:
        tbl['contig_length'] = [float(clens[i]) for i in tbl['contig']]
    except:
        tbl['contig_length'] = float(clens["_".join(i.split("_")[])])

    hits_cols = [i for i in tbl.columns if 'hit' in i]
    count_cols = ["_".join(["reads",i.split("_")[1]]) for i in hits_cols]

    for h, c in zip(hits_cols, count_cols):
        fr = tbl[h]/(tbl[c] * tbl['contig_length']) * mult
        tbl[h.replace("hit_","fr_")] = fr
    return tbl

In [31]:
clens

{'AG-414-G13_AG-414-G13_NODE_1': 335021,
 'AG-414-G13_AG-414-G13_NODE_10': 4063,
 'AG-414-G13_AG-414-G13_NODE_11': 3460,
 'AG-414-G13_AG-414-G13_NODE_2': 304226,
 'AG-414-G13_AG-414-G13_NODE_3': 179074,
 'AG-414-G13_AG-414-G13_NODE_4': 100922,
 'AG-414-G13_AG-414-G13_NODE_5': 79580,
 'AG-414-G13_AG-414-G13_NODE_6': 51285,
 'AG-414-G13_AG-414-G13_NODE_7': 39754,
 'AG-414-G13_AG-414-G13_NODE_8': 9181,
 'AG-414-G13_AG-414-G13_NODE_9': 4234}

In [ ]:
def construct_recruit_tbl(vir_tsv, bac_tsv, read_count_dict, contig_file):
    '''
    Args:
        vir_tsv: diamond recruitment converted to tsv for vir metagenome
        bac_tsv: diamond recruitment converted to tsv for bac metagenome
        read_count_dict: dict of mg read counts with two keys -- 'vir_reads' and 'bac_reads'
        contig_file: path to a file with sag contigs in it; either in fasta or gff format
    Returns:
        pandas dataframe with mg fraction calculated
    '''
    cnames = "qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore".split()
    bac_df = import_diamond_tsv(bac_tsv)
    vir_df = import_diamond_tsv(vir_tsv)

    bac_sum = pd.Series(bac_df.groupby('sseqid')['qseqid'].count(), name='hit_mg-bac')
    vir_sum = pd.Series(vir_df.groupby('sseqid')['qseqid'].count(), name='hit_mg-vir')

    orfhits = pd.concat([bac_sum, vir_sum], axis=1).reset_index().rename(columns={'index':'orf'})
    orfhits = map_orfs_to_contigs(orfhits, contig_file)

    chits = pd.concat([summarize_by_contig(orfhits, 'hit_mg-bac'), summarize_by_contig(orfhits, 'hit_mg-vir')], axis=1)
    chits['reads_mg-vir'] = float(read_count_dict['vir_reads'])
    chits['reads_mg-bac'] = float(read_count_dict['bac_reads'])

    clens = contig_lengths(contig_file)

    out_tbl = compute_fr(chits.reset_index(), clens, mult=1e6)
    out_tbl['ratio_virus_bacteria'] = out_tbl['fr_mg-vir'] / out_tbl['fr_mg-bac']
    out_tbl['ratio_virus_bacteria'] = [1000 if i == float('inf') else i for i in out_tbl['ratio_virus_bacteria']]

    return out_tbl

In [ ]:
cnames = "qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore".split()
bac_df = import_diamond_tsv(bac_tsv)
vir_df = import_diamond_tsv(vir_tsv)